In [100]:
import os
import sys
import random
import itertools
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import Bio.PDB.Polypeptide as pp
from collections import Counter
from getmutations import MutationsDict, GetMutations
%matplotlib inline

In [2]:
AA = list(pp.aa1)

In [3]:
figures_path = "../../../Dropbox/perturbation_networks/draft/figures"

In [4]:
DATA = 'data/'

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
mpl.style.use('seaborn-colorblind')
mpl.rcParams['grid.color'] = 'xkcd:white'
mpl.rcParams['grid.linestyle'] = '-'
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams['figure.facecolor'] = 'xkcd:white'
mpl.rcParams["axes.facecolor"] = 'xkcd:white'
mpl.rcParams["savefig.facecolor"] = 'xkcd:white'

### Functional Data

In [79]:
proteins = ['1be9', '1d5r', '1nd4', '3dqw', '4bz3']
protein_names = ['PSD95', 'PTEN', 'APH(3\')II', 'Src CD', 'VIM-2']
to_names = {i:j for (i,j) in zip(proteins, protein_names)}

In [7]:
# Import processed functional data as DataFrames, all files have ordered AA list as index, positions as columns
# Save data in functional_data
functional_data = dict()
for protein in proteins:
    csv_file = os.path.join(DATA, f'functional_{protein}.csv')
    functional_data[protein] = pd.read_csv(csv_file, index_col=0, header=0)

### Perturbation Network Data and Related Functions

In [12]:
data_path = os.path.join(DATA, 'structure')
thresholds = [round(i, 1) for i in np.linspace(3, 10, 71)]
sample_thresholds = [round(i, 1) for i in np.linspace(3, 10, 8)]
measures = ['nodes', 'edges', 'weight', 'distance']

In [46]:
def ReadNetworkCSV(protein, threshold, measure):
    """Return DataFrame from corresponding CSV. If protein has multiple identical chains, return average value for 
    each position amongst all chains."""
    file = os.path.join(os.path.join(DATA, 'structure'), f"{protein}/{protein}_{threshold}_{measure}.csv")
    network_df = pd.read_csv(file, header=0)
    network_df.index = AA
    # Get chains from columns
    column_names = list(network_df.columns)
    chains = list(set([position[1] for position in column_names]))
    # Get positions without chain distinction from functional files
    positions = list(functional_data[protein].columns)
    average = pd.DataFrame(index=AA, columns=positions, dtype=np.float64)
    # Save data for position over chains in list, write average into df
    for position in positions:
        for aa in AA:
            values = []
            for chain in chains:
                check = position[0]+chain+position[1:]
                if check in network_df.columns:
                    values.append(network_df.at[aa, check])
            if values:
                average_value = sum(values)/len(values)
                average.at[aa, position] = average_value
    return average

In [10]:
def Standardize(protein, threshold, measure):
    """Return standardized values from network data. Make 0's into NaN. """
    network_df = ReadNetworkCSV(protein, threshold, measure)
    for position in network_df.columns:
        for aa in network_df.index:
            if position[0] == aa:
                network_df.at[aa, position] = np.nan
    data_array = network_df.to_numpy()
    data_mean = np.nanmean(network_df, dtype=np.float64)
    data_std = np.nanstd(network_df, dtype=np.float64)
    network_df = network_df.apply(lambda x:(x-data_mean)/data_std)
    return network_df 

In [11]:
def GetPercentage(percentage, which, data, return_score=False):
    """Return set with top or bottom percentage of positions according to functional data. 
    Parameters:
        percentage (float): between 0 and 1, percentage of positions that we want.
        which (str): 'highest', 'lowest'
        data (dataframe): functional data to consider mean of
        return_score (bool): If True, return list of tuples with mean value and position
    Returns:
        Set of positions.
    """
    functional_mean = data.mean()
    positions = list(data.columns)
    pairs = [(functional_mean[pos], pos) for pos in positions] 
    pairs.sort(key = lambda x:x[0]) 
    if which == 'highest': 
        pairs.reverse() 
    n = int(len(positions)*percentage)
    if return_score:
        return [pair for pair in pairs[:n]]
    else:
        return set([pair[1] for pair in pairs[:n]])

In [15]:
def GetNetworkExtremes(protein, mincount, measure_cutoffs, thresh=9.0):
    """ Return set with positions that pass measure sd cutoffs for at least mincount measures. """
    network_extremes_list = []
    for i,measure in enumerate(measures): 
        threshold = 3.8 if measure == 'distance' else thresh
        network_df = Standardize(protein, threshold, measure)
        if measure_cutoffs[i] > 0:
            extremes = network_df.columns[(network_df > measure_cutoffs[i]).any()].tolist()
        else:
            extremes = network_df.columns[(network_df < measure_cutoffs[i]).any()].tolist()
        network_extremes_list.extend(extremes)

    counter = Counter(network_extremes_list)
    positions = list(set(network_extremes_list))
    return set([pos for pos in positions if counter[pos] >= mincount])

In [16]:
def ToPercentage(a,b):
    """Return percentage form of a/b, if b != 0. If given set or list, use len of. 
    If string, return formatted percentage, else float."""
    x = a if type(a) == int or type(a) == float else len(a)
    y = b if type(b) == int or type(b) == float else len(b)
    
    if y == 0:
        return np.nan
    else:
        return round(100*x/y,1)

## Predictions to test 

### Structurally Sensitive Positions

In [44]:
def ComparePredictionsLoss(functional_percentage, network_mincount, measure_cutoffs=[1,1,1,1], thresh=9.0, 
                           string=True):
    """Compare percentage of positions with highest mean functional values with predicted positions above cutoff for 
    perturbation network data. Return True Positives, False Positives, Coverage, Accuracy, and percentages of 
    positions. 
    """
    predict = pd.DataFrame(index=proteins, columns=['True Positives', 'False Positives', 'Coverage', 'Accuracy',
                                                          'Prediction %', 'Functional %'])
    for protein in proteins:
        network_extremes = GetNetworkExtremes(protein, network_mincount, measure_cutoffs, thresh=thresh)
        funct_extremes = GetPercentage(functional_percentage, 'lowest', functional_data[protein])
        
        how_many = len(network_extremes.intersection(funct_extremes))
        positions = len(functional_data[protein].columns)

        predict.at[protein, 'True Positives'] = how_many
        predict.at[protein, 'False Positives'] = len(network_extremes) - how_many 
        predict.at[protein,'Coverage']= ToPercentage(how_many, funct_extremes)
        predict.at[protein, 'Accuracy'] = ToPercentage(how_many, network_extremes)
        predict.at[protein,'Functional %'] = ToPercentage(funct_extremes, positions)
        predict.at[protein,'Prediction %'] = ToPercentage(network_extremes, positions)
     
    percentages = ['Coverage', 'Accuracy','Prediction %', 'Functional %']
    
    for score in percentages:
        total = 0
        for protein in proteins:
            total += predict.at[protein, score]
        predict.at['Mean', score] = ToPercentage(total,500)
        
    if string: 
        return predict.style.format({col:'{0:,.1f}%' for col in percentages})
    else:
        return predict 

In [152]:
def ComparePredictionsGain(functional_percentage, network_mincount, measure_cutoffs=[1,1,1,1], thresh=9.0, 
                          string=True):
    """Compare percentage of positions with highest mean functional values with complement of predicted positions for 
    loss of function for given measure_cutoffs and mincounts. Return True Positives, False Positives, Coverage, 
    Accuracy, and percentages of positions. 
    """
    predict = pd.DataFrame(index=proteins, columns=['True Positives', 'False Positives', 'Coverage', 'Accuracy',
                                                          'Prediction %', 'Functional %'])
    for protein in proteins:
        network_extremes_loss = GetNetworkExtremes(protein, network_mincount, measure_cutoffs, thresh=thresh)
        total_positions = functional_data[protein].columns
        funct_extremes = GetPercentage(functional_percentage, 'highest', functional_data[protein])
        network_extremes = set([pos for pos in total_positions if pos not in network_extremes_loss])
        
        how_many = len(network_extremes.intersection(funct_extremes))
        positions = len(functional_data[protein].columns)

        predict.at[protein, 'True Positives'] = how_many
        predict.at[protein, 'False Positives'] = len(network_extremes) - how_many 
        predict.at[protein,'Coverage']= ToPercentage(how_many, funct_extremes)
        predict.at[protein, 'Accuracy'] = ToPercentage(how_many, network_extremes)
        predict.at[protein,'Functional %'] = ToPercentage(funct_extremes, positions)
        predict.at[protein,'Prediction %'] = ToPercentage(network_extremes, positions)
        
    percentages = ['Coverage', 'Accuracy','Prediction %', 'Functional %']
    
    for score in percentages:
        total = 0
        for protein in proteins:
            total += predict.at[protein, score]
        predict.at['Mean', score] = ToPercentage(total,500)
        
    if string: 
        return predict.style.format({col:'{0:,.1f}%' for col in percentages})
    else:
        return predict 

In [48]:
headers = ['Maximizing Accuracy', 'Maximizing Coverage', 'Maximizing Both']
scores = ['Accuracy', 'Coverage', 'Prediction %']
index = pd.MultiIndex.from_product([headers, scores], names=['Prediction', 'Score'])

In [50]:
loss_predictions = pd.DataFrame(index=protein_names, columns=index)

In [51]:
accuracy = ComparePredictionsLoss(0.4, 4, [1.5,1.5,1.5,1.5], string=False)
coverage = ComparePredictionsLoss(0.4, 2, [1,1,1,1], string=False)
both = ComparePredictionsLoss(0.4, 2, [1.5,1.5,1.5,1.5], string=False)
best_loss = [accuracy, coverage, both]

for j,prediction in enumerate(headers):
    for i, protein in enumerate(protein_names):
        for score in scores:
            loss_predictions.at[protein, (prediction, score)] = best_loss[j].at[proteins[i], score]

for prediction in headers:
    for score in scores:
        total = 0
        for protein in protein_names:
            total += loss_predictions.at[protein, (prediction, score)]
        loss_predictions.at['Mean', (prediction, score)] = ToPercentage(total,500)

In [154]:
gain_predictions = pd.DataFrame(index=protein_names, columns=index)

In [155]:
accuracy = ComparePredictionsGain(0.4, 1, [1,1,1,1], string=False)
coverage = ComparePredictionsGain(0.4, 3, [1,1,1,1], string=False)
both = ComparePredictionsGain(0.4, 2, [1,1,1,1], string=False)
best_gain = [accuracy, coverage, both]

for j,prediction in enumerate(headers):
    for i, protein in enumerate(protein_names):
        for score in scores:
            gain_predictions.at[protein, (prediction, score)] = best_gain[j].at[proteins[i], score]

for prediction in headers:
    for score in scores:
        total = 0
        for protein in protein_names:
            total += gain_predictions.at[protein, (prediction, score)]
        gain_predictions.at['Mean', (prediction, score)] = ToPercentage(total,500)

## Testing for statistical significance / null model

In [17]:
def PredictRandom(protein, percentage):
    '''Select percentage of positions at random.'''
    positions = list(functional_data[protein].columns)
    n = int(len(positions)*percentage)
    predictions = np.random.choice(positions, n, replace=False)
    return set(predictions)

In [147]:
def TestRandom(functional_percentage, prediction_percentage, runs, plot=False):
    '''Test Coverage and Accuracy of random predictions. '''
    labels1 = ['PSD95', 'PTEN', 'APH(3\')II', 'Src CD', 'VIM-2']
    df = pd.DataFrame(columns=['Mean', 'SD', 'Type', 'Protein'])
    
    ###### vamos a querer los plots??? o lo quito??
    if plot:
        fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20,9), sharey=True)

        fig.add_subplot(211, frameon=False)
        plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
        plt.xlabel('Accuracy (%)')

        fig.add_subplot(212, frameon=False)
        plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
        plt.xlabel('Coverage (%)')

        plt.subplots_adjust(hspace=0.2)
    
    if type(prediction_percentage) is float: # if given same value for 5 proteins
        prediction_percentage = [prediction_percentage]*5
        
    for i, protein in enumerate(proteins):
        coverage, accuracy = [], []
        n = len(functional_data[protein].columns)
        funct_extremes = GetPercentage(functional_percentage, 'lowest', functional_data[protein])
        
        for j in range(runs):
            predictions = PredictRandom(protein, prediction_percentage[i])
            how_many = len(predictions.intersection(funct_extremes))
            coverage.append(ToPercentage(how_many, funct_extremes))
            accuracy.append(ToPercentage(how_many, predictions))
        
        ## normal?
        mu_c = np.mean(coverage)
        sigma_c = np.std(coverage)
        x = np.linspace(0, 100, 101)

        mu_a = np.mean(accuracy)
        sigma_a = np.std(accuracy)
        x = np.linspace(0, 100, 101)
            
        ## Save mean and std 
        df.loc[len(df)] = [mu_c, sigma_c, 'Coverage', protein_names[i]]
        df.loc[len(df)] = [mu_a, sigma_a, 'Accuracy', protein_names[i]]
        
        if plot:
            axes[1,i].set_xlim(0,100)
            axes[0,i].set_xlim(0,100)
            sns.kdeplot(np.array(coverage), ax=axes[1,i], bw_adjust=2, color='darkblue')
            sns.kdeplot(np.array(accuracy), ax=axes[0,i], bw_adjust=2, color='darkgreen')
            axes[1,i].plot(x, sp.stats.norm.pdf(x, mu_c, sigma_c))
            axes[0,i].plot(x, sp.stats.norm.pdf(x, mu_a, sigma_a))

            axes[0,i].set_title(protein_names[i])
        
    mean_coverage = df.loc[df['Type'] == 'Coverage'].mean()
    df.loc[len(df)] = [mean_coverage[0], mean_coverage[1], 'Coverage', 'Mean']
    mean_accuracy = df.loc[df['Type'] == 'Accuracy'].mean()
    df.loc[len(df)] = [mean_accuracy[0], mean_accuracy[1], 'Accuracy', 'Mean']
    
    if plot: 
        return fig, df
    else: 
        return df

### Testing Structurally Sensitive Predictions

In [82]:
# Summary of loss predictions 
loss_predictions

Prediction Maximizing Accuracy                       Maximizing Coverage  \
Score                 Accuracy Coverage Prediction %            Accuracy   
PSD95                     66.7     42.4         25.3                60.9   
PTEN                        84     55.7         26.4                66.9   
APH(3')II                 90.5     37.3         16.5                65.9   
Src CD                    72.5     38.5         21.1                  51   
VIM-2                     90.7     53.3         23.4                65.9   
Mean                      80.9     45.4         22.5                62.1   

Prediction                       Maximizing Both                        
Score      Coverage Prediction %        Accuracy Coverage Prediction %  
PSD95          84.8         55.4            68.4     78.8         45.8  
PTEN             91         54.1            74.3     82.8         44.3  
APH(3')II      83.3         50.6            73.1     66.7         36.5  
Src CD         80.2         62.4            62.3     68.8         43.8  
VIM-2          96.7         58.4            73.1     85.9         46.8  
Mean           87.2         56.2            70.2     76.6         43.4

In [160]:
def get_pvalues(prediction_df, prediction):
    """Return dataframe with scores, z-scores, and p values for z-test to random data from 10,000 runs, based on 
    predictions from loss_predictions or gain_predictions. 
    
    Parameters:
        prediction (str): Maximizing Accuracy, Coverage or Both 
        prediction_df (DataFrame): loss_predictions or gain_predictions
    
    Returns:
        data (DataFrame), with data for coverage and accuracy for five proteins and mean. 
    """
    # prediction percentages are mean from these predictions
    percentages = list(prediction_df[(prediction, 'Prediction %')]
                                   .apply(lambda x:round(x/100,3)))[:-1]
    # Functional percentage is 0.4
    data = TestRandom(0.4, percentages, 10000)
    
    # Add scores from predictions
    data.loc[data['Type'] == 'Accuracy', 'Score'] = prediction_df[(prediction, 'Accuracy')].tolist()
    data.loc[data['Type'] == 'Coverage', 'Score'] = prediction_df[(prediction, 'Coverage')].tolist()
    
    # Add z scores 
    data['Z score'] = data.apply(lambda x: (x['Score']-x['Mean'])/x['SD'], axis=1)
    
    ### z-test 
    # Null hypothesis is that coverage and accuracy scores are obtained from seen prediction percentage through random
    # predictions. 
    data['p value'] = data.apply(lambda x: sp.stats.norm.sf(abs(-x['Z score'])), axis=1)
    return data

In [162]:
loss_predictions

Prediction Maximizing Accuracy                       Maximizing Coverage  \
Score                 Accuracy Coverage Prediction %            Accuracy   
PSD95                     66.7     42.4         25.3                60.9   
PTEN                        84     55.7         26.4                66.9   
APH(3')II                 90.5     37.3         16.5                65.9   
Src CD                    72.5     38.5         21.1                  51   
VIM-2                     90.7     53.3         23.4                65.9   
Mean                      80.9     45.4         22.5                62.1   

Prediction                       Maximizing Both                        
Score      Coverage Prediction %        Accuracy Coverage Prediction %  
PSD95          84.8         55.4            68.4     78.8         45.8  
PTEN             91         54.1            74.3     82.8         44.3  
APH(3')II      83.3         50.6            73.1     66.7         36.5  
Src CD         80.2         62.4            62.3     68.8         43.8  
VIM-2          96.7         58.4            73.1     85.9         46.8  
Mean           87.2         56.2            70.2     76.6         43.4

In [161]:
get_pvalues(loss_predictions, 'Maximizing Accuracy')

,Mean,SD,Type,Protein,Score,Z score,p value
0,24.124100,5.879959,Coverage,PSD95,42.4,3.108168,9.412547e-04
1,39.808000,9.708663,Accuracy,PSD95,66.7,2.769897,2.803698e-03
2,26.372330,3.078517,Coverage,PTEN,55.7,9.526558,8.129265e-22
3,39.720080,4.642668,Accuracy,PTEN,84.0,9.537601,7.308721e-22
4,16.441420,2.847768,Coverage,APH(3')II,37.3,7.324536,1.198630e-13
5,39.948470,6.916515,Accuracy,APH(3')II,90.5,7.308815,1.347544e-13
6,21.039430,3.243320,Coverage,Src CD,38.5,5.383549,3.651569e-08
7,39.597830,6.108618,Accuracy,Src CD,72.5,5.386189,3.598363e-08
8,23.391970,3.445346,Coverage,VIM-2,53.3,8.680705,1.966623e-18
9,39.837910,5.873582,Accuracy,VIM-2,90.7,8.659468,2.369860e-18


In [163]:
get_pvalues(loss_predictions, 'Maximizing Coverage')

,Mean,SD,Type,Protein,Score,Z score,p value
0,54.174820,6.736606,Coverage,PSD95,84.8,4.546085,2.732654e-06
1,39.731610,4.938392,Accuracy,PSD95,60.9,4.286495,9.075723e-06
2,54.071780,3.519078,Coverage,PTEN,91.0,10.493720,4.616005e-26
3,39.758540,2.603859,Accuracy,PTEN,66.9,10.423553,9.679212e-26
4,50.540770,3.850653,Coverage,APH(3')II,83.3,8.507447,8.890225e-18
5,39.965380,3.044972,Accuracy,APH(3')II,65.9,8.517194,8.173245e-18
6,62.377990,3.818230,Coverage,Src CD,80.2,4.667611,1.523611e-06
7,39.653900,2.426179,Accuracy,Src CD,51.0,4.676531,1.458841e-06
8,58.001730,3.962222,Coverage,VIM-2,96.7,9.766811,7.814752e-23
9,39.826000,2.711567,Accuracy,VIM-2,65.9,9.615842,3.427317e-22


In [164]:
get_pvalues(loss_predictions, 'Maximizing Both')

,Mean,SD,Type,Protein,Score,Z score,p value
0,45.810070,6.716891,Coverage,PSD95,78.8,4.911488,4.519396e-07
1,39.772380,5.841521,Accuracy,PSD95,68.4,4.900713,4.774471e-07
2,44.324920,3.510660,Coverage,PTEN,82.8,10.959500,2.991454e-28
3,39.762030,3.145100,Accuracy,PTEN,74.3,10.981519,2.344860e-28
4,36.492580,3.662127,Coverage,APH(3')II,66.7,8.248599,8.013116e-17
5,40.020760,4.017136,Accuracy,APH(3')II,73.1,8.234533,9.012978e-17
6,43.462880,3.934479,Coverage,Src CD,68.8,6.439765,5.982942e-11
7,39.732140,3.607434,Accuracy,Src CD,62.3,6.255931,1.975754e-10
8,46.794280,4.047484,Coverage,VIM-2,85.9,9.661736,2.191858e-22
9,39.857660,3.446927,Accuracy,VIM-2,73.1,9.644052,2.604512e-22


In [165]:
get_pvalues(gain_predictions, 'Maximizing Accuracy')

,Mean,SD,Type,Protein,Score,Z score,p value
0,28.849160,6.189300,Coverage,PSD95,54.5,4.144385,1.703638e-05
1,39.671640,8.506595,Accuracy,PSD95,72.0,3.800388,7.223488e-05
2,28.986770,3.176363,Coverage,PTEN,50.0,6.615500,1.851496e-11
3,39.739530,4.352739,Accuracy,PTEN,67.8,6.446623,5.718490e-11
4,30.598370,3.571915,Coverage,APH(3')II,57.8,7.615419,1.314184e-14
5,40.003930,4.670074,Accuracy,APH(3')II,75.6,7.622163,1.247296e-14
6,24.382880,3.399756,Coverage,Src CD,39.6,4.475945,3.803708e-06
7,39.687780,5.550019,Accuracy,Src CD,64.4,4.452637,4.241107e-06
8,26.440200,3.536843,Coverage,VIM-2,53.3,7.594287,1.547460e-14
9,39.874360,5.341287,Accuracy,VIM-2,79.0,7.325134,1.193301e-13


In [166]:
get_pvalues(gain_predictions, 'Maximizing Coverage')

,Mean,SD,Type,Protein,Score,Z score,p value
0,49.467110,6.782521,Coverage,PSD95,69.7,2.983093,1.426758e-03
1,39.810590,5.467491,Accuracy,PSD95,54.8,2.741552,3.057484e-03
2,52.165430,3.496457,Coverage,PTEN,80.3,8.046595,4.256481e-16
3,39.776030,2.664804,Accuracy,PTEN,60.9,7.927026,1.122284e-15
4,59.130870,3.828896,Coverage,APH(3')II,89.2,7.853212,2.027583e-15
5,39.940990,2.588433,Accuracy,APH(3')II,59.9,7.710847,6.249271e-15
6,46.270350,3.974291,Coverage,Src CD,65.6,4.863672,5.761379e-07
7,39.665570,3.418967,Accuracy,Src CD,56.2,4.836090,6.620908e-07
8,50.128920,4.107939,Coverage,VIM-2,85.9,8.707793,1.549239e-18
9,39.758530,3.262337,Accuracy,VIM-2,67.5,8.503557,9.193393e-18


In [167]:
get_pvalues(gain_predictions, 'Maximizing Both')

,Mean,SD,Type,Protein,Score,Z score,p value
0,44.468610,6.775649,Coverage,PSD95,69.7,3.723834,9.811008e-05
1,39.621840,6.048236,Accuracy,PSD95,62.2,3.733015,9.460048e-05
2,45.621250,3.498906,Coverage,PTEN,73.8,8.053588,4.020077e-16
3,39.757430,3.048101,Accuracy,PTEN,63.8,7.887720,1.538781e-15
4,49.017630,3.859767,Coverage,APH(3')II,84.3,9.141062,3.092104e-20
5,39.998000,3.150750,Accuracy,APH(3')II,68.3,8.982622,1.321947e-19
6,37.202370,3.826625,Coverage,Src CD,55.2,4.703264,1.280174e-06
7,39.682800,4.072943,Accuracy,Src CD,58.2,4.546393,2.728656e-06
8,41.535080,3.976502,Coverage,VIM-2,75.0,8.415667,1.953328e-17
9,39.805890,3.811472,Accuracy,VIM-2,71.9,8.420397,1.876059e-17
